# Advent of Code Day 9

On Day 9, the objective is to build a character stream parser to handle some special cases and compute a score based on the stream.  In part one, the goal is to count the number of groups (indicated by {}), giving each a bonus based on its depth (i.e. being inside another group).   However, <> are garbage and no groups can appear within them and ! cancel the next character.  In part two, we need to also count the number of cancelled characters. 

In [ ]:
from utils import read_input

### Parser

This is the first problem where I thought making a class made sense.  This implementation works correctly, but it's readability is not great.  I could nominally improve it by breaking up the parse method into other methods, but this feels like a case where the State pattern should apply.  In other languages, I'd define an inner State class for the various modes and it would have access to the variables in the containing scope.  However, Python doesn't support it in that way so I need to figure out the Pythonic way to approach it.  I could expose some parts of Parser as "public" methods and have State manipulate those, but that would send the wrong message about Parser.  Since the State methods are pretty unique to the Parser, I could have the States simply manipulate the Parser's "private" instances as needed.  This is probably what I'll do and I think it's on the right side of Python's intent, but this is a case where Pyton is quite a bit different from other languages I've used so I need to find the right idiom.

In [ ]:
from __future__ import print_function

class Parser(object):
    
    def __init__(self, verbose = False):
     
        self._current_depth = 0               
        self._score = 0
        self._garbage_mode = False
        self._ignore_mode = False
        self._cancelled = 0
        if verbose:
            self._print = lambda x: print(x)
        else:
            self._print = lambda x: ()
    
    def parse(self, s):
        for c in s:
            self._print('Processing {}'.format(c))
            # If the last character was a !, we always ignore the current character
            if self._ignore_mode:
                print('Ignore mode enabled so ignoring {} and turning off ignore'.format(c))               
                self._ignore_mode = False
            elif c == '!':
                print("Starting ignore mode as ! was read")
                self._ignore_mode = True            
            elif c == '<' and not self._garbage_mode:
                self._garbage_mode = True
                self._print('Start garbage mode')
            elif c == '>':
                self._garbage_mode = False  
                self._print('End garbage mode')
            elif c == '{' and not self._garbage_mode:            
                self._current_depth +=1
                self._print('Increasing depth to {}'.format(self._current_depth))
            elif c == '}' and not self._garbage_mode:
                self._print('Adding {} to score'.format(self._current_depth))
                self._score += self._current_depth
                self._print('Score now equal to {}'.format(self._score))
                self._current_depth -= 1
                self._print('Decreasing depth to {}'.format(self._current_depth))
            elif self._garbage_mode:
                self._print('Incrementing garbage mode count as {} is during garbage mode'.format(c))
                self._cancelled += 1                
            else:
                self._print('No action taken on character {}'.format(c))           
            
    def reset(self):
        self._current_depth = 0
        self._score = 0        
        self._garbage_mode = False
        self._ignore_mode = False
        self._cancelled = 0
        
    def cancelled_characters(self):
        return self._cancelled
    
    def score(self):
        return self._score

        
class _NormalParseMode:
    
    def __init__(self):
        pass
    
    def next(c):
        pass
        

### Parser Tests

It made sense to define some actual tests for this particular problem given the complexity.

In [ ]:
def run_parse_tests():
    p = Parser(verbose = True)
    
    test_cases = [(1, '{}'), (6, '{{{}}}'), (5, '{{},{}}'), (16, '{{{},{},{{}}}}'),
                  (1, '{<a>,<a>,<a>,<a>}'), (9, '{{<ab>},{<ab>},{<ab>},{<ab>}}'), 
                  (9, '{{<!!>},{<!!>},{<!!>},{<!!>}}'), (3, '{{<a!>},{<a!>},{<a!>},{<ab>}}')]
    
    for i, (expected, test_string) in enumerate(test_cases):
        print('Running test {}'.format(i + 1))
        
        p.parse(test_string)
        assert expected == p.score(), 'Expected {} != Actual {}'.format(expected, actual)
        print('Test Complete\n')
        p.reset()
        
    print("All tests passed")
    
def run_cancellation_count_tests():
    p = Parser(verbose = True)
    
    test_cases = [(0, '<>'), (17, '<random characters>'), (3, '<<<<>'), (2, '<{!>}>'),
                  (0, '<!!>'), (0, '<!!!>>'), (10, '<{o"i!a,<{i<a>')]
    
    for i, (expected, test_string) in enumerate(test_cases):
        print('Running test {}'.format(i + 1))
        
        p.parse(test_string)
        assert expected == p.cancelled_characters(), 'Expected {} != Actual {}'.format(expected, actual)
        print('Test Complete\n')
        p.reset()
        
    print("All tests passed")

In [ ]:
def solve_both_parts():
    stream = read_input('Input/day9.txt')[0]
    
    parser = Parser(verbose = False)
    parser.parse(stream)
    
    print('Score = {}'.format(parser.score()))
    print('Cancelled = {}'.format(parser.cancelled_characters()))


In [ ]:
solve_both_parts()